# Step 1 - Scrap Wikipedia for data

In [1]:

#install Beautiful Soup and requests for Web Scaping
!pip install BeautifulSoup4
!pip install requests

# Step 2: Get Data

In [3]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())

    if (index == 0):
        columns = section
    else:
        data.append(section)

canadapostcode_df = pd.DataFrame(data = data,columns = columns)
canadapostcode_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Step 3 : Clean Data

In [6]:
canadapostcode_df = canadapostcode_df[canadapostcode_df['Borough'] != 'Not assigned']
canadapostcode_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
canadapostcode_df["Neighbourhood"] = canadapostcode_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))
canadapostcode_df = canadapostcode_df.drop_duplicates()
if(canadapostcode_df.index.name != 'Postcode'):
   canadapostcode_df = canadapostcode_df.set_index('Postcode') 
canadapostcode_df.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


In [8]:
canadapostcode_df['Neighbourhood'].replace("Not assigned", canadapostcode_df["Borough"],inplace=True)
canadapostcode_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


# Step 3:  Number of Rows of the Data Frame


In [10]:
canadapostcode_df.shape

(103, 2)